# Cut Columns
This notebook is designed to reformat the column order and number of variables so that the training and predict data sets used with a given SuperLearner model are consistent. This allows the ML model to be reused across multiple predict data sets.

In [1]:
import pandas as pd
import numpy as np

# Load data

In [2]:
# Initial Training data
training_data_df = pd.read_csv('../../sbir-learnerworks-doc/notebooks/whondrml_all.csv')

# Predict data
predict_data_df = pd.read_csv('../../sbir-learnerworks-doc/GLORICH_HydroSHEDS/step_12_output.csv')

# WHONDRS data updates
update_training_df = pd.read_csv('../../sbir-learnerworks-doc/GLORICH_HydroSHEDS/step_11_output.csv')

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (79) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Remove columns

In [3]:
# Remove any WHONDRS metadata not available in global training data.
# Also remove any features we know we don't want (e.g. lon,lat).

training_data_df.drop(columns=[
    'US_Latitude_dec.deg',
    'US_Water.Column.Height_cm',
    'US_Algal.Mat.Coverage',
    'US_Macrophyte.Coverage',
    'US_Sunlight.Access_Perc.Canopy.Cover',
    'Stream_Order',
    'NOSC',
    'lamO2',
    'lamHCO3',
    'n_chems',
    'skew_lamO2',
    'skew_lamHCO3'],inplace=True)

training_data_df.drop(columns=[x for x in training_data_df.columns if "US_Depositional.Type_" in x],inplace=True)
training_data_df.drop(columns=[x for x in training_data_df.columns if "Sediment_" in x],inplace=True)
training_data_df.drop(columns=[x for x in training_data_df.columns if "General_Vegetation_" in x],inplace=True)
training_data_df.drop(columns=[x for x in training_data_df.columns if "perc_" in x],inplace=True)
training_data_df.drop(columns=[x for x in training_data_df.columns if "del_" in x],inplace=True)
training_data_df.drop(columns=[x for x in training_data_df.columns if "Hydrogeomorphology_" in x],inplace=True)
training_data_df.drop(columns=[x for x in training_data_df.columns if "River_Gradient_" in x],inplace=True)

# Process the update_training data so that it has the same columns as the original training data.
# Note some additional steps are required below.
# TO DO: replace the steps here and the steps below with a bash script for automated production.
update_training_df.drop(columns=[
    'GL_id',
    'GL_lon',
    'GL_lat',
    'STAT_ID',
    'TOC',
    'RA_lm',
    'my_lm',
    'RA_lon',
    'RA_lat',
    'STAT_ID',
    'dist_m'],inplace=True)

# In the process of dropping lon,lat from predictions,
# store them for plotting later.
xy_df = pd.DataFrame()
xy_df['GL_id'] = predict_data_df.pop('GL_id')
xy_df['lon'] = predict_data_df.pop('GL_lon')
xy_df['lat'] = predict_data_df.pop('GL_lat')

# Drop variables that we will not use in the ML from the predict data.
predict_data_df.drop(columns=[
    'RA_lm',
    'my_lm',
    'RA_lon',
    'RA_lat',
    'STAT_ID',
    'dist_m',
    'TOC'
    ],inplace=True)

# Note here and below we shift to amplitude of stream flow change.
predict_data_df['RA_ms_av'] = predict_data_df['RA_cms_cyr']/predict_data_df['RA_xam2']
predict_data_df['RA_ms_di'] = (predict_data_df['RA_cms_cmx'] - predict_data_df['RA_cms_cmn'])/predict_data_df['RA_xam2']

update_training_df['RA_ms_av'] = update_training_df['RA_cms_cyr']/update_training_df['RA_xam2']
update_training_df['RA_ms_di'] = (update_training_df['RA_cms_cmx'] - update_training_df['RA_cms_cmn'])/update_training_df['RA_xam2']

predict_data_df.drop(columns=[
    'RA_cms_cyr',
    'RA_cms_cmn',
    'RA_cms_cmx',
    'RA_xam2'
    ],inplace=True)

update_training_df.drop(columns=[
    'RA_cms_cyr',
    'RA_cms_cmn',
    'RA_cms_cmx',
    'RA_xam2'
    ],inplace=True)

# Reorder some columns
training_data_df['Temp_water'] = training_data_df.pop('SW_Temp_degC')
training_data_df['pH'] = training_data_df.pop('SW_pH')
training_data_df['DO_mgL'] = training_data_df.pop('DO_mg.per.L')
training_data_df['DOSAT'] = training_data_df.pop('DO_perc.sat')
training_data_df['RA_ms_av'] = training_data_df.pop('RA_ms_av')
training_data_df['RA_ms_di'] = training_data_df.pop('RA_ms_mx') - training_data_df.pop('RA_ms_mn')
training_data_df['rate.mg.per.L.per.h'] = training_data_df.pop('rate.mg.per.L.per.h')

update_training_df['rate.mg.per.L.per.h'] = update_training_df.pop('rate.mg.per.L.per.h')

# Append the training data to the update (that way we can keep the data sets separate for now)
update_training_df = update_training_df.append(training_data_df)

In [4]:
# Both training and predict data are consistent, 
# but need to remove more features for clarity.

# Get rid of any upstream info
training_data_df.drop(columns=[x for x in training_data_df.columns if "_u" in x],inplace=True)
predict_data_df.drop(columns=[x for x in predict_data_df.columns if "_u" in x],inplace=True)
update_training_df.drop(columns=[x for x in update_training_df.columns if "_u" in x],inplace=True)

# All the anthropogenic influencers are correlated, so pick one for now:
drop_anthro_features = ['pop_ct_csu','urb_pc_cse','nli_ix_cav','rdd_mk_cav','hft_ix_c09','gdp_md_cav','hdi_ix_cav']
training_data_df.drop(columns=drop_anthro_features,inplace=True)
predict_data_df.drop(columns=drop_anthro_features,inplace=True)
update_training_df.drop(columns=drop_anthro_features,inplace=True)

# Get rid of some more variables
drop_features = [
    'pac_pc_cse',
    'cly_pc_cav',
    'slt_pc_cav',
    'snd_pc_cav',
    'soc_th_cav',
    'kar_pc_cse',
    'ero_kh_cav',
    'swc_pc_cyr',
    'swc_pc_cdi',
    'aet_mm_cyr',
    'aet_mm_cdi',
    'cmi_ix_cyr',
    'cmi_ix_cdi',
    'snw_pc_cyr',
    'snw_pc_cmx',
]
training_data_df.drop(columns=drop_features,inplace=True)
predict_data_df.drop(columns=drop_features,inplace=True)
update_training_df.drop(columns=drop_features,inplace=True)

# Check column order (essential for SuperLearner)

In [5]:
ii = 0
for col in predict_data_df.columns:
    print(str(ii)+'  '+col+'  '+training_data_df.columns[ii]+'  '+update_training_df.columns[ii])
    ii = ii + 1

0  RA_SO  RA_SO  RA_SO
1  RA_dm  RA_dm  RA_dm
2  run_mm_cyr  run_mm_cyr  run_mm_cyr
3  dor_pc_pva  dor_pc_pva  dor_pc_pva
4  gwt_cm_cav  gwt_cm_cav  gwt_cm_cav
5  ele_mt_cav  ele_mt_cav  ele_mt_cav
6  slp_dg_cav  slp_dg_cav  slp_dg_cav
7  sgr_dk_rav  sgr_dk_rav  sgr_dk_rav
8  tmp_dc_cyr  tmp_dc_cyr  tmp_dc_cyr
9  tmp_dc_cdi  tmp_dc_cdi  tmp_dc_cdi
10  pre_mm_cyr  pre_mm_cyr  pre_mm_cyr
11  pre_mm_cdi  pre_mm_cdi  pre_mm_cdi
12  for_pc_cse  for_pc_cse  for_pc_cse
13  crp_pc_cse  crp_pc_cse  crp_pc_cse
14  pst_pc_cse  pst_pc_cse  pst_pc_cse
15  ire_pc_cse  ire_pc_cse  ire_pc_cse
16  gla_pc_cse  gla_pc_cse  gla_pc_cse
17  prm_pc_cse  prm_pc_cse  prm_pc_cse
18  ppd_pk_cav  ppd_pk_cav  ppd_pk_cav
19  Temp_water  Temp_water  Temp_water
20  pH  pH  pH
21  DO_mgL  DO_mgL  DO_mgL
22  DOSAT  DOSAT  DOSAT
23  RA_ms_av  RA_ms_av  RA_ms_av
24  RA_ms_di  RA_ms_di  RA_ms_di


In [6]:
xy_df

,GL_id,lon,lat
0,300043,0.002154,41.135430
1,300086,0.092954,41.680723
2,300022,-0.122750,41.927435
3,300023,-0.131028,41.918934
4,300153,0.144293,41.122866
...,...,...,...
10524,5,-68.445497,44.640715
10525,6,-76.884050,37.571283
10526,7,-88.655746,33.430906
10527,8,-88.433519,33.453633


# Write output files

In [7]:
training_data_df.to_csv('whondrml_global_train_25_inputs.csv',index=False,na_rep='NaN')
update_training_df.to_csv('whondrml_global_train_25_inputs_update.csv',index=False,na_rep='NaN')
predict_data_df.to_csv('WH_RA_GL_global_predict_25_inputs.csv',index=False,na_rep='NaN')
xy_df.to_csv('WH_RA_GL_global_predict_ixy.csv',index=False,na_rep='NaN')